# 04 Linear algebra

## Vectors


Abstractly, vectors are objects that can be added together to form new vectors and that can be multiplied by scalars (i.e., numbers), also to form new vectors.

Concretely (for us), vectors are points in some finite-dimensional space.
Although you might not think of your data as vectors, they are often a
useful way to represent numeric data.
For example, if you have the heights, weights, and ages of a large
number of people, you can treat your data as three-dimensional vector
`[height, weight, age]`
. If you’re teaching a class with four exams, you can treat student
grades as four-dimensional vectors `[exam1, exam2, exam3, exam4]`.

The simplest from-scratch approach is to represent vectors as lists of numbers. A list of three numbers corresponds to a vector in three- dimensional space, and vice versa.

We’ll accomplish this with a type alias that says a `Vector` is just a
`list` of `float`s:

In [1]:
from typing import List

Vector = List[float]

height_weight_age = [
    70,  # inches
    170, # pounds
    40   # years
]

grades = [
    95,  # exam1
    80,  # exam2
    75,  # exam3
    62   # exam4
]

We’ll also want to perform arithmetic on vectors.
Because Python `list`s aren’t vectors (and hence provide no facilities for vector arithmetic), we’ll need to build these arithmetic tools ourselves. So let’s start with that.

We can easily implement this by `zip`-ing the vectors together and using a list comprehension to add the corresponding elements:

In [4]:
def add(v: Vector, w: Vector) -> Vector:
    """Adds corresponding elements"""
    assert len(v) == len(w), 'vectors must be the same length'

    return [v_i + w_i for v_i, w_i in zip(v, w)]

assert add([1, 2, 3], [4, 5, 6]) == [5, 7, 9], 'This is an example of holding the addition definition'

Similarly, to subtract two vectors we just subtract the corresponding elements:

In [5]:
def subtract(v: Vector, w: Vector) -> Vector:
    """Sbtracts corresponding elements"""
    assert len(v) == len(w), 'vectors must be the same length'
    
    return [v_i - w_i for v_i, w_i in zip(v, w)]

assert subtract([5, 7, 9], [4, 5, 6]) == [1, 2, 3], 'This is an example of holding the subtract definition'

We’ll also sometimes want to componentwise sum a list of vectors—that is, create a new vector whose first element is the sum of all the first elements, whose second element is the sum of all the second elements, and so on:

In [12]:
def vector_sum(vectors: List[Vector]) -> Vector:
    """Sums all corresponding elements"""
    # check that vectors is not empty
    assert vectors, 'no vectors provided.'
    
    # check the vectors are all the same size
    num_elements = len(vectors[0])
    assert all(len(v) == num_elements for v in vectors), 'different sizes!'
    
    # the i-th element of the result is the sum of every vector[i]
    return [sum(vector[i] for vector in vectors)
                              for i in range(0, num_elements)]

assert vector_sum([[1, 2], [3, 4], [5, 6], [7, 8]]) == [16, 20]
assert vector_sum([[1, 2, 3], [4, 5, 6]]) == [5, 7, 9]

We’ll also need to be able to multiply a vector by a scalar, which we do simply by multiplying each element of the vector by that number:

In [13]:
def scalar_multiply(c: float, v: Vector) -> Vector:
    """Multiplies every element vy c"""
    return [c * v_i for v_i in v]

assert scalar_multiply(2, [1, 2, 3]) == [2, 4, 6]

This allows us to compute the componentwise means of a list of (same- sized) vectors:

In [15]:
def vector_mean(vectors: List[Vector]) -> Vector:
    """Computes the element-wise average"""
    n = len(vectors)
    return scalar_multiply(1 / n, vector_sum(vectors))

assert vector_mean([[1, 2], [3, 4], [5, 6]]) == [3, 4]

A less obvious tool is the dot product. The dot product of two vectors is the sum of their componentwise products:

In [17]:
def dot(v: Vector, w: Vector) -> float:
    """Computes v_1 * w_1 + ... + v_n * w_n"""
    assert len(v) == len(w), 'vectors must be the same length'
    
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

assert dot([1, 2, 3], [4, 5, 6]) == 32

If has magnitude 1, the dot product measures how far the vector extends
in the direction. For example, if `w = [1, 0]` , then 
`dot(v, w)` is just the first component
of `v`. Another way of saying this is that it’s the length of the
vector you’d get if you projected `v` onto `w`.

Using this, it’s easy to compute a vector’s sum of squares:

In [18]:
def sum_of_squares(v: Vector) -> float:
    """Return v_1 * v_1 + ... + v_n * v_n"""
    return dot(v, v)

assert sum_of_squares([1, 2, 3]) == 14

which we can use to compute its magnitude (or length):

In [19]:
import math

def magnitude(v: Vector) -> float:
    """Returns the magnitude (or length) of v"""
    return math.sqrt(sum_of_squares(v))

assert magnitude([3, 4]) == 5

We now have all the pieces we need to compute the distance between two vectors, defined as:


$$ \sqrt{(v_1 - w_1)^2 + ... + (v_n -w_n)^2}$$

In code:

In [20]:
def squared_distance(v: Vector, w: Vector) -> float:
    """Computes (v_1 - w_1) ** 2 + ... + (v_n - w_n) ** 2"""
    return sum_of_squares(subtract(v, w))

def distance(v: Vector, w: Vector) -> float:
    """Computes the distance between v and w"""
    return math.sqrt(squared_distance(v, w))


This is possibly clearer if we write it as (the equivalent):

In [21]:
def distance(v: Vector, w: Vector) -> float:
    return magnitude(subtract(v, w))

## Matrices

A matrix is a two-dimensional collection of numbers. We will represent matrices as lists of lists, with each inner list having the same size and representing a row of the matrix. If `A` is a matrix, then `A[i][j]` is the element in the ith row and the jth column. Per mathematical convention, we will frequently use capital letters to represent matrices. 

In [23]:
# Another type alias
Matrix = List[List[float]]

A = [
    [1, 2, 3],
    [4, 5, 6]
]

B = [
    [1, 2],
    [3, 4],
    [5, 6]
]

Given this list-of-liss representation, the matrix `A` has `len(A)` rows and `len(A[0])`
columns, which we consider its `shape`:

In [25]:
from typing import Tuple

def shape(A: Matrix) -> Tuple[int, int]:
    """Returns (# of rows of A, # of columns of A)"""
    num_rows = len(A)
    num_cols = len(A[0]) if A else 0
    return num_rows, num_cols

assert shape([[1, 2, 3], [4, 5, 6]]) == (2, 3)

If a matrix has n rows and k columns, we will refer to it as an n × k matrix. We can (and sometimes will) think of each row of an n × k matrix as a vector of length k, and each column as a vector of length n:

In [26]:
def get_row(A: Matrix, i: int) -> Vector:
    """Returns the i-th row of A (as a Vector)"""
    return A[i]


def get_colum(A: Matrix, j:int) -> Vector:
    """Returns the j-th column of A (as a Vector)"""
    return [A_i[j] for A_i in A]

We’ll also want to be able to create a matrix given its shape and a function for generating its elements. We can do this using a nested list comprehension:|

In [34]:
from typing import Callable

def make_matrix(num_rows:int, num_cols:int, entry_fn:Callable[[int, int], float]) -> Matrix:
    """Returns a num_rows x num_cols matrix
    whose (i, j)-th entry is entry_fn(i, j)
    """
    return [
        [entry_fn(i, j) for j in range(0, num_cols)]       # create list [entry_fn(i, 0), ..., entry_fn(i, num_cols-1)]
                            for i in range(0, num_rows)  # create one list for each i (num_rows - 1)
    ]

Given this function, you could make a 5 × 5 identity matrix (with 1s on the diagonal and 0s elsewhere) like so:

In [36]:
def identity_matrix(n:int) -> Matrix:
    """Returns the n x n identity matrix"""
    return make_matrix(n, n, lambda i, j: 1 if i == j else 0)

assert identity_matrix(5) == [
    [1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1]
]

Matrices will be important to us for several reasons.

First, we can use a matrix to represent a dataset consisting of multiple vectors, simply by considering each vector as a row of the matrix. For example, if you had the heights, weights, and ages of 1,000 people, you could put them in a 1,000 × 3 matrix:



Second, as we’ll see later, we can use an n × k matrix to represent a linear function that maps k-dimensional vectors to n-dimensional vectors. Several of our techniques and concepts will involve such functions.


Third, matrices can be used to represent binary relationships. 

In [37]:
friendships = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
               (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

We could also represent this as:

In [38]:
friend_matrix = [[0, 1, 1, 0, 0, 0, 0, 0, 0, 0],  # user 0
                 [1, 0, 1, 1, 0, 0, 0, 0, 0, 0],  # user 1
                 [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],  # user 2
                 [0, 1, 1, 0, 1, 0, 0, 0, 0, 0],  # user 3
                 [0, 0, 0, 1, 0, 1, 0, 0, 0, 0],  # user 4
                 [0, 0, 0, 0, 1, 0, 1, 1, 0, 0],  # user 5
                 [0, 0, 0, 0, 0, 1, 0, 0, 1, 0],  # user 6
                 [0, 0, 0, 0, 0, 1, 0, 0, 1, 0],  # user 7
                 [0, 0, 0, 0, 0, 0, 1, 1, 0, 1],  # user 8
                 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]  # user 9

If there are very few connections, this is a much more inefficient representation, since you end up having to store a lot of zeros. However, with the matrix representation it is much quicker to check whether two nodes are connected—you just have to do a matrix lookup instead of (potentially) inspecting every edge:

In [39]:
assert friend_matrix[0][2] == 1, "0 and 2 are friends"
assert friend_matrix[0][8] == 0, "0 and 8 are not friends"

Similarly, to find a node’s connections, you only need to inspect the column (or the row) corresponding to that node:

In [40]:
# only need to look at one row
friends_of_five = [i for i, is_friend in enumerate(friend_matrix[5]) if is_friend]

friends_of_five

[4, 6, 7]

With a small graph you could just add a list of connections to each node object to speed up this process; but for a large, evolving graph that would probably be too expensive and difficult to maintain.
